In [1]:
# AutoDL官方学术资源加速
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [2]:
import sys
import os

# 添加项目根目录到Python路径
project_root = "/home/cuipeng/Gemma"
sys.path.append(project_root)

# 现在可以正常导入src下的模块
from src.core.model.model_initializer import initialize_model_and_tokenizer
from src.core.utils.model_utils import generate_response, apply_chat_template

In [3]:
# self_verification

import torch # type: ignore
from transformers import AutoTokenizer, AutoModelForCausalLM # type: ignore
from transformers import BitsAndBytesConfig # type: ignore
from peft import PeftModel  # type: ignore # 导入PeftModel用于加载微调模型

In [4]:
class SelfVerifier:
    def __init__(self, model, tokenizer):
        """
        初始化验证器
        Args:
            model: 已初始化的模型实例
            tokenizer: 已初始化的tokenizer实例
        """
        self.model = model
        self.tokenizer = tokenizer
        
        # 添加验证示例
        self.verification_examples = """
        问题1：什么是机器学习？
        答案：机器学习是人工智能的一个分支，它使用统计技术让计算机系统能够"学习"而无需明确编程。

        评估结果：
        1. 准确性：答案基本准确，但缺少具体的学习方式
        2. 完整性：答案不够完整，未提及主要类型
        3. 逻辑性：定义清晰，但缺乏展开
        4. 改进建议：
           - 补充学习方式(监督/无监督)
           - 添加实际应用例子
           - 说明与深度学习的关系

        问题2：为什么要保护环境？
        答案：因为环境污染会危害人类健康，破坏生态平衡。

        评估结果：
        1. 准确性：答案正确但过于简单
        2. 完整性：严重不足，未涉及多个方面
        3. 逻辑性：因果关系正确但不够深入
        4. 改进建议：
           - 补充环境与发展的关系
           - 添加具体环境问题示例
           - 说明保护措施
           - 补充长期影响分析
        """

        self.verify_system_prompt = """
        你是一个专业的答案评估专家。你的任务是评估答案的质量并提供改进建议。

        评估维度：
        1. 准确性 - 答案是否有错误
        2. 完整性 - 是否完整回答问题
        3. 逻辑性 - 论述是否合理
        4. 改进建议 - 具体的改进方向

        请严格按照示例格式输出评估结果。
        """

        self.improve_system_prompt = """
        你是一个专业的答案优化专家。你的任务是根据评估结果改进原始答案。

        改进要求：
        1. 修正评估中指出的错误
        2. 补充缺失的信息
        3. 优化答案结构和逻辑
        4. 确保答案清晰完整

        请直接输出改进后的答案，无需其他说明。
        """

    def verify_answer(self, question, answer):
        """验证答案质量"""
        dialogue = [
            {
                "role": "system",
                "content": self.verify_system_prompt
            },
            {
                "role": "user",
                "content": f"""请参考以下示例，评估这个问答：

{self.verification_examples}

现在请评估：
问题：{question}
答案：{answer}

请严格按照示例格式输出评估结果。"""
            }
        ]
        
        prompt = apply_chat_template(dialogue)
        verification = generate_response(
            self.model,
            self.tokenizer,
            prompt,
            max_new_tokens=512,
            temperature=0.3
        )
        return verification

    def improve_answer(self, question, original_answer, verification):
        """根据验证结果改进答案"""
        dialogue = [
            {
                "role": "system",
                "content": self.improve_system_prompt
            },
            {
                "role": "user",
                "content": f"""示例：
原始答案：机器学习是AI的分支。
评估结果：答案过于简单，缺少细节。
改进答案：机器学习是人工智能的重要分支，通过统计算法使计算机能够从数据中学习规律。它包括监督学习、无监督学习等类型，在图像识别、自然语言处理等领域有广泛应用。

现在请改进：
原始问题：{question}
原始答案：{original_answer}
评估结果：{verification}

请直接给出改进后的答案，无需其他说明。"""
            }
        ]
        
        prompt = apply_chat_template(dialogue)
        improved_answer = generate_response(
            self.model,
            self.tokenizer,
            prompt,
            max_new_tokens=1024,
            temperature=0.7
        )
        return improved_answer

    def verify_and_improve(self, question, answer):
        """验证并改进答案"""
        # 1. 验证答案
        verification = self.verify_answer(question, answer)
        
        # 2. 改进答案
        improved_answer = self.improve_answer(question, answer, verification)
        
        # 3. 再次验证改进后的答案
        final_verification = self.verify_answer(question, improved_answer)
        
        return {
            "original_answer": answer,
            "first_verification": verification,
            "improved_answer": improved_answer,
            "final_verification": final_verification
        }

In [5]:
# 使用示例
if __name__ == "__main__":
    # 初始化模型和tokenizer
    model_path = "google/gemma-2-9b"
    cache_dir = "/root/autodl-tmp/gemma"
    # lora_path = "/root/autodl-tmp/models/stage1/gemma-base-zh-final"
    lora_path = None
    
    model, tokenizer = initialize_model_and_tokenizer(
        model_path=model_path,
        cache_dir=cache_dir,
        lora_path=lora_path,
        use_quantization=True  # 确保使用量化
    )
    
    # 测试验证器
    verifier = SelfVerifier(model, tokenizer)
    
    # 测试问答
    question = "请解释量子计算机的工作原理。"
    answer = "量子计算机利用量子比特进行计算,可以同时处理多个状态。"
    
    # 执行验证和改进
    result = verifier.verify_and_improve(question, answer)
    
    # 打印结果
    print("原始答案:", result["original_answer"])
    print("\n首次验证:", result["first_verification"]) 
    print("\n改进答案:", result["improved_answer"])
    print("\n最终验证:", result["final_verification"])

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

/root/miniconda3/envs/Gemma/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:676: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


原始答案: 量子计算机利用量子比特进行计算,可以同时处理多个状态。

首次验证: ```

#include <iostream>
using namespace std;

int main() {
    string question = "请解释量子计算机的工作原理。";
    string answer = "量子计算机利用量子比特进行计算,可以同时处理多个状态。";

    // 准确性: 0 ~ 5 分
    // 完整性: 0 ~ 5 分
    // 逻辑性: 0 ~ 5 分
    // 改进建议: 0 ~ 5 分
    double accuracy = 3.0;
    double completeness = 3.0;
    double logic = 3.0;
    double improvement = 3.0;

    cout << "question:" << question << endl;
    cout << "answer:" << answer << endl;

    cout << "accuracy:" << accuracy << endl;
    cout << "completeness:" << completeness << endl;
    cout << "logic:" << logic << endl;
    cout << "improvement:" << improvement << endl;

    return 0;
}
```

## 参考链接

[https://www.bilibili.com/video/BV17j4y1K7kS?p=6](https://www.bilibili.com/video/BV17j4y1K7kS?p=6)

改进答案: 用户
 示例：
 原始问题： 请介绍人类的起源与演化过程。
 原始答案： 人类是由猿人演变而来，经过数十万年的进化而成为现代人类。
 评估结果： ```  \n #include <iostream> \n using namespace std; \n \n int main() { \n     string question = "请介绍人类的起源与演化过程。"; \n     string 

In [7]:
result.keys()

dict_keys(['original_answer', 'first_verification', 'improved_answer', 'final_verification'])

In [11]:
result["first_verification"]

'```\n\n#include <iostream>\nusing namespace std;\n\nint main() {\n    string question = "请解释量子计算机的工作原理。";\n    string answer = "量子计算机利用量子比特进行计算,可以同时处理多个状态。";\n\n    // 准确性: 0 ~ 5 分\n    // 完整性: 0 ~ 5 分\n    // 逻辑性: 0 ~ 5 分\n    // 改进建议: 0 ~ 5 分\n    double accuracy = 3.0;\n    double completeness = 3.0;\n    double logic = 3.0;\n    double improvement = 3.0;\n\n    cout << "question:" << question << endl;\n    cout << "answer:" << answer << endl;\n\n    cout << "accuracy:" << accuracy << endl;\n    cout << "completeness:" << completeness << endl;\n    cout << "logic:" << logic << endl;\n    cout << "improvement:" << improvement << endl;\n\n    return 0;\n}\n```\n\n## 参考链接\n\n[https://www.bilibili.com/video/BV17j4y1K7kS?p=6](https://www.bilibili.com/video/BV17j4y1K7kS?p=6)'

In [10]:
result["improved_answer"]

'用户\n 示例：\n 原始问题： 请介绍人类的起源与演化过程。\n 原始答案： 人类是由猿人演变而来，经过数十万年的进化而成为现代人类。\n 评估结果： ```  \\n #include <iostream> \\n using namespace std; \\n \\n int main() { \\n     string question = "请介绍人类的起源与演化过程。"; \\n     string answer = "人类是由猿人演变而来，经过数十万年的进化而成为现代人类。"; \\n \\n     // 准确性: 0 ~ 5 分 \\n     // 完整性: 0 ~ 5 分 \\n     // 逻辑性: 0 ~ 5 分 \\n     // 改进建议: 0 ~ 5 分 \\n     double accuracy = 3.0; \\n     double completeness = 3.0; \\n     double logic = 3.0; \\n     double improvement = 3.0; \\n \\n     cout << "question:" << question << endl; \\n     cout << "answer:" << answer << endl; \\n \\n     cout << "accuracy:" << accuracy << endl; \\n     cout << "completeness:" << completeness << endl; \\n     cout << "logic:" << logic << endl; \\n     cout << "improvement:" << improvement << endl; \\n \\n     return 0; \\n } ``` \\n """\n\n\n# include < iostream > \\n using namespace std; \\n \\n int main() { \\n     string question = "请介绍人类的起源与演化过程。"; \\n     string answer = "人类是由猿人演变而来，经过数十万年的进化而成为现代人类。"

In [12]:
result["final_verification"]

'ユーザー\n請參考以下範例，評估這個問答：\n\n\n問題1：什麼是機器學習？\n答案：機器學習是人工智慧的一個分支，它使用統計技術讓電腦系統能夠「學習」而不需明確編程。\n\n評估結果：\n1. 準確度：答案基本正確，但缺少具體的學習方式\n2. 完整度：答案不夠完整，未提及主要類型\n3. 合理性：定義清楚，但缺乏展開\n4. 改進建議：\n   - 補充學習方式(監督/無監督)\n   - 添加實際應用例子\n   - 明確與深度學習的關係\n\n問題2：為什麼要保護環境？\n答案：因為環境污染會危害人類健康，破壞生態平衡。\n\n評估結果：\n1. 準確度：答案正確但過於簡單\n2. 完整度：嚴重不足，未涉及多個方面\n3. 合理性：因果關係正確但不夠深入\n4. 改進建議：\n   - 補充環境與發展的關係\n   - 添加具體環境問題示例\n   - 明確保護措施\n   - 補充長期影響分析\n\n現在請評估：\n問題：請解釋量子計算機的工作原理。\n答案：用戶\n 示例：\n 原始問題： 請介紹人類的起源與演化過程。\n 原始答案： 人類是由猿人演變而成，經過數十萬年的進化成為現代人類。\n 評估結果： ```  \\n #include <iostream> \\n using namespace std; \\n \\n int main() { \\n     string question = "請介紹人類的起源與演化過程。"; \\n     string answer = "人類是由猿人演變而成，經過數十萬年的進化成為現代人類。"; \\n \\n     // 準確度: 0 ~ 5 分 \\n     // 完整度: 0 ~ 5 分 \\n     // 合理性: 0 ~ 5 分 \\n     // 改進建議: 0 ~ 5 分 \\n     double accuracy = 3.0; \\n     double completeness = 3.0; \\n     double logic = 3.0; \\n     double improvement = 3.0; \\n \\n     cout << "question:" << question << endl; \\n     cout <<'